## eager API

In [2]:
from __future__ import print_function
import tensorflow as tf
import tensorflow.contrib.eager as tfe

In [3]:
tfe.enable_eager_execution()

In [5]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [6]:
# Parameters
learning_rate = 0.001
num_steps = 1000
batch_size = 128
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

In [10]:
dataset = tf.data.Dataset.from_tensor_slices(
    (mnist.train.images, mnist.train.labels)).batch(batch_size)
dataset_iter = tfe.Iterator(dataset)

In [14]:
# 定义神经网络， 使用eager API 和tf.layers API
class NeuralNet(tfe.Network):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.layer1 = self.track_layer(tf.layers.Dense(n_hidden_1, activation=tf.nn.relu))
        self.layer2 = self.track_layer(tf.layers.Dense(n_hidden_2, activation=tf.nn.relu))
        self.out_layer = self.track_layer(tf.layers.Dense(num_classes))
    
    def call(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return self.out_layer(x)


In [15]:
neural_net = NeuralNet()

In [16]:
# 定义交叉熵
def loss_fn(inference_fn, inputs, labels):
    # 使用sparse_softmax 交叉熵
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=inference_fn(inputs), labels=labels))

In [17]:
# 计算预测精确度
def accuracy_fn(inference_fn, inputs, labels):
    prediction = tf.nn.softmax(inference_fn(inputs))
    correct_pred = tf.equal(tf.arg_max(prediction, 1), labels)
    return tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [18]:
# 随机梯度下降优化
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
grad = tfe.implicit_gradients(loss_fn)

In [19]:
# 训练
average_loss = 0
average_acc = 0
for step in range(num_steps):
    try:
        d = dataset_iter.next()
    except StopIteration:
        dataset_iter = tfe.Iterator(dataset)
        d = dataset_iter.next()
    
    x_batch = d[0]
    y_batch = tf.cast(d[1], dtype=tf.int64)
    
    batch_loss = loss_fn(neural_net, x_batch, y_batch)
    average_loss += batch_loss
    
    # 计算batch的精确度
    batch_accuracy = accuracy_fn(neural_net, x_batch, y_batch)
    average_acc += batch_accuracy
    
    if step == 0:
        print("Initial loss= {:.9f}".format(average_loss))
    optimizer.apply_gradients(grad(neural_net, x_batch, y_batch))
    # Display info
    if (step + 1) % display_step == 0 or step == 0:
        if step > 0:
            average_loss /= display_step
            average_acc /= display_step
        print("Step:", '%04d' % (step + 1), " loss=",
              "{:.9f}".format(average_loss), " accuracy=",
              "{:.4f}".format(average_acc))
        average_loss = 0.
        average_acc = 0.


Instructions for updating:
Use `argmax` instead
Initial loss= 2.365533829
Step: 0001  loss= 2.365533829  accuracy= 0.1016
Step: 0100  loss= 0.599711597  accuracy= 0.8245
Step: 0200  loss= 0.251560390  accuracy= 0.9282
Step: 0300  loss= 0.217466012  accuracy= 0.9356
Step: 0400  loss= 0.180566877  accuracy= 0.9452
Step: 0500  loss= 0.135868117  accuracy= 0.9611
Step: 0600  loss= 0.119870178  accuracy= 0.9627
Step: 0700  loss= 0.112093724  accuracy= 0.9674
Step: 0800  loss= 0.106556065  accuracy= 0.9670
Step: 0900  loss= 0.086113781  accuracy= 0.9738
Step: 1000  loss= 0.077219829  accuracy= 0.9755


In [20]:
# Evaluate model on the test image set
testX = mnist.test.images
testY = mnist.test.labels

test_acc = accuracy_fn(neural_net, testX, testY)
print("Testset Accuracy: {:.4f}".format(test_acc))

Testset Accuracy: 0.9715
